In [1]:
%load_ext autoreload
%autoreload 2

from plot_fuse_results import (plot_results2, get_merged_table, get_bohlin78)
from matplotlib import pyplot as plt
from astropy.table import Column

plt.ioff()

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 250
#%matplotlib widget

KeyboardInterrupt: 

# Get the data

### data for Gordon09 extinction curve sightlines

In [ ]:
data = get_merged_table()

# add comments for certain stars here
data.add_column(Column(['no'] * len(data), dtype='<U16', name='comment'))
def set_comment(name, s):
    data['comment'][data['Name'] == name] = s

for name in ["HD200775", "HD164906", "HD045314", "HD206773"]:
    set_comment(name, "lo_h_av")

set_comment("HD096675", "hi_h_av")

# write out in fits format so we can look at it in topcat
data.write('merged_table.fits', overwrite=True)
#data

### data for Gordon09 comparision star sightlines

In [ ]:
data_comp = get_merged_table(comp=True)

### data for Bohlin 78 Copernicus sightlines

In [ ]:
data_bohlin78 = get_bohlin78()

# 1. Basic gas-to-dust slopes
## Gas-to-A(V)
- 4 strong outlieres in NH-AV relation. Including them breaks the fit. Effect on NH-E(B-V) is not that strong.
- Need to investigate these points, as they might also have a strong effect on the fit result of other plots.
- gas-to-A(V) narrower than gas-to-E(B-V)

In [ ]:
fig1 = plot_results2(data, 'AV', 'nhtot', pxrange=[0.0,3.5], pyrange=[0.0,0.8e22],
            data_comp=data_comp, data_bohlin=data_bohlin78)
fig2 = plot_results2(data, 'AV', 'nhtot', pxrange=[0.0,3.5], pyrange=[0.0,0.8e22],
            data_comp=data_comp, data_bohlin=data_bohlin78, ignore_comments=['lo_h_av', 'hi_h_av'])

In [ ]:
fig1

In [ ]:
fig2

## Gas-to-E(B-V)


In [ ]:
fig1 = plot_results2(data, 'EBV', 'nhtot', pyrange=[0.0,0.8e22],
            data_comp=data_comp, data_bohlin=data_bohlin78)
fig2 = plot_results2(data, 'EBV', 'nhtot', pyrange=[0.0,0.8e22],
            data_comp=data_comp, data_bohlin=data_bohlin78, ignore_comments=['lo_h_av', 'hi_h_av'])

In [ ]:
fig1

In [ ]:
fig2

# 2. Gas-to-dust vs other things

## VS dust column and gas column

Before making any big conlusions here, the correlation between NH / AV and AV needs to be implemented properly. Preliminarily, we have:

### Using A(V)
The bad points mentioned above seem to pull down NH/AV - AV pretty badly. Not including them makes the downward slope insignificant. 

With them included:

NH-AV goes down with AV, but up with NH! Despite the fact that AV and NH have a well-defined slope in the plots above!

In [ ]:
#print(data.columns)
fig1 = plot_results2(data, 'AV', 'NH_AV', pyrange=[0.0,0.5e22], pxrange=[0,4],
            data_comp=data_comp,data_bohlin=data_bohlin78)
fig2 = plot_results2(data, 'nhtot', 'NH_AV', pyrange=[0.0,0.5e22], pxrange=[0, 1.2e22],
            data_comp=data_comp,data_bohlin=data_bohlin78)

In [ ]:
fig1

In [ ]:
fig2

### Using E(B-V)

In this case, the slope of NH / EBV is unclear, but NH / EBV does sem to go up with EBV!

In [ ]:
fig1 = plot_results2(data, 'EBV', 'NH_EBV', pyrange=[0.0,1.5e22], pxrange=[-.1,1],
            data_comp=data_comp,data_bohlin=data_bohlin78)
fig2 = plot_results2(data, 'nhtot', 'NH_EBV', pyrange=[0.0,1.5e22], pxrange=[0,1.2e22],
            data_comp=data_comp,data_bohlin=data_bohlin78)

In [ ]:
fig1

In [ ]:
fig2

## VS average grain size (RV)

In [ ]:
plot_results2(data, 'RV', 'NH_AV', pxrange=[2.5,6.0], pyrange=[0.0,0.5e22],
            data_comp=data_comp)

## VS molecular fraction (fH2)
### Using A(V)

In [ ]:
plot_results2(data, 'fh2', 'NH_AV', pxrange=[0.0,0.7], pyrange=[0.0,0.5e22],
            data_comp=data_comp, data_bohlin=data_bohlin78)

### Using E(B-V)

In [ ]:
plot_results2(data, 'fh2', 'NH_EBV', pxrange=[0.0,0.7], pyrange=[0.0,1.5e22],
            data_comp=data_comp, data_bohlin=data_bohlin78)

# 3. Molecular fraction vs others 

E(B-V) seems a better predictor of fh2 than A(V)

## VS E(B-V) dust column

In [ ]:
plot_results2(data, 'EBV', 'fh2',
            data_comp=data_comp, data_bohlin=data_bohlin78)

## VS A(V) dust column

In [ ]:
plot_results2(data, 'AV', 'fh2',
            data_comp=data_comp, data_bohlin=data_bohlin78)

## VS average grain size (RV)

In [ ]:
plot_results2(data, 'RV', 'fh2', pxrange=[2.0,6.0],
            data_comp=data_comp, ignore_comments=['lo_h_av', 'hi_h_av'])

In [ ]:
plot_results2(data, 'EBV', 'RV')

In [ ]:
plot_results2(data, 'AV', 'RV')

# Other ideas

longitude versus rv and distance versus rv (and versus fh2)

COS FUV needs 2x10-13 ergs/blah blah

E140H for CO (maybe E140M), S/N 15...

E(B-V)/distance versus fH2.  Should also do A(V)/d.  Maybe n(H) as well.